# KEN3140 (Semantic Web) Lab 9
### Constructing ontologies with the OWL API

**Date:** 29 September 2021

**Author:** Kody Moodley

**Affiliation:** Institute of Data Science, Maastricht University

**License:** [GNU Affero General Public License v3.0](https://www.gnu.org/licenses/agpl-3.0.txt)

**Notebook description**

This is a Jupyter notebook to which allows one to programmatically construct ontologies using the [Java-based OWL API](https://github.com/owlcs/owlapi/wiki/Documentation) without requiring knowledge of the full OWL API. I have built a Java wrapper for the OWLAPI called [SimpleOWLAPI](https://github.com/kodymoodley/simpleowlapi). This wrapper allows you to make use of the most important features of the OWL API in a much more concise way than having to use the original API.

**Background: please read this section carefully before beginning the lab**

This lab follows on from Lab 6, 7, and 8 in which you familiarised yourself with the intuitions behind constructing OWL ontologies, the expressivity, reasoning complexity and semantics that need to be taken into account when constructing an ontology, and how this can affect its accuracy or quality. So far, you have used [Protege](https://protege.stanford.edu/) for constructing ontologies. Protege is a well-known ontology editor and it also has a cloud-based, collaborative version - [WebProtege](https://webprotege.stanford.edu/). However, Protege is just one tool amongst a variety of Semantic Web tools, not only ontology editors, but also reasoners, web applications and frameworks that push the vision of the Semantic Web forward. This lab will ask you to construct an OWL ontology **programmatically** using the OWL API instead of using a graphical ontology editor.

*Why not just use a graphical ontology editor?*

You can! However, in practice, it is useful to be able to load an ontology, edit it and "attach" it to extracted RDF data all programmatically from a script or software library. This is so that we can perform queries, reasoning, and mining algorithms that form part of a complex chain of computational processes (i.e., a group of software services called a scientific workflow).

**Lab instructions** 

Your task in this lab is to construct an OWL ontology describing the familiar domain of family relations that we have been using throughout this course. In other words, build **your own** family relations ontology with OWL 2 DL axioms that **you** think should be in this ontology. You do not have to reproduce the same ontology that was used in Labs 6-8. However, you may reuse some terms from this ontology if you wish. You will find example ontology construction commands you will need in Sections 2-3 of this notebook. Make use of the reasoner **incrementally** after adding new axioms (especially complex ones such as role constraints axioms) to check if unexpected inferences or logical errors arise. Example commands of how to invoke the reasoner can be found in Section 3. Please build **your** ontology in Section 4 of this notebook.

**Notebook structure**

* Sections 1 - 3 of this notebook demonstrates in detail, through examples, how to use SimpleOWLAPI to build OWL 2 DL ontologies programmatically
* Section 4 is where you will actually construct your own ontology using commands such as those in Sections 1 - 3

**Learning objectives**

How to programmatically:

* Construct and reason with OWL 2 DL ontologies
* Generate explanations for entailments in OWL 2 DL ontologies
* Debug unintended inferences in OWL 2 DL ontologies

**Technical requirements**

* Jupyter notebooks: **https://jupyter.org/**
* IJava: **https://github.com/SpencerPark/IJava** (Java kernel for Jupyter notebooks)

**Documentation**

For more details about how to use SimpleOWLAPI, consult the [documentation](https://github.com/kodymoodley/simpleowlapi) on Github and the [Javadocs](https://kodymoodley.github.io/simpleowlapi/doc/) for the library. The original OWL API documentation can be found here: **https://github.com/owlcs/owlapi/wiki/Documentation**. The Javadocs for the full OWL API version 4 are located here: **http://owlcs.github.io/owlapi/apidocs_4/** and those for OWL API version 5 are located here: **http://owlcs.github.io/owlapi/apidocs_5/**.

### 1. Import [SimpleOWLAPI](https://github.com/kodymoodley/simpleowlapi)
First we need to import the [SimpleOWLAPI](https://github.com/kodymoodley/simpleowlapi) Java library. This library is maintained by [Kody Moodley](https://github.com/kodymoodley).

In [ ]:
%jars simpleowlapi-lib-1.2.1-jar-with-dependencies.jar

In [ ]:
%%loadFromPOM
<dependency>
  <groupId>org.slf4j</groupId>
  <artifactId>slf4j-nop</artifactId>
  <version>1.7.30</version>
</dependency>

In [ ]:
import org.semanticweb.owl.simpleowlapi.*;
import org.semanticweb.owlapi.model.*;

### 2. Examples to get started
Have a look at these examples to show you how to build ontologies using SimpleOWLAPI

#### 2a. Creating an ontology

In [ ]:
// create a new SimpleOWLAPIFactory instance which allows the construction and manipulation of OWL ontologies (default OWL reasoner is JFACT)
SimpleOWLAPIFactory s = new SimpleOWLAPIFactory(); 
// create a new OWL ontology by specifying an IRI string and set it to the currently selected (active) ontology
s.createOntology("http://maastrichtuniversity.nl/semweb/2021/lab9#");

#### 2b. Creating (atomic) classes

In [ ]:
// create single atomic class
s.createClass("Woman");
// create multiple atomic classes (each one separated by a space)
s.createClasses("Man Male Female Person Adult Car Tyre USCitizen Passport Leg Arthropod JohnsBookClubMembers");
// print out the axioms and entities in the ontology
s.printOntology();

#### 2c. Creating properties

In [ ]:
// create single object property
s.createObjectProperty("hasGender");
// create multiple object properties at once (each separated by a space)
s.createObjectProperties("hasPart teaches studentOf owns hasSister hasSibling hasDocument");
// specify whether an object property is transitive, reflexive, symmetric etc.
// parameter1: name of the property
// parameter2: transitive? 0 - no, 1 - yes
// parameter3: reflexive? 0 - no, 1 - yes, 2 - irreflexive
// parameter4: symmetric? 0 - no, 1 - yes, 2 - antisymmetric
s.createObjectProperty("marriedTo", 0, 2, 1);
// alternatively, if you have already created a property
//s.makeTransitive("propertyName");
// similar process for DATA properties
s.createDataProperties("hasWeight name bornOn"); 
s.printOntology();

#### 2d. Creating individuals

In [ ]:
// create single individual
s.createIndividual("john");
// create multiple individuals at once (each separatedby a space)
s.createIndividuals("sara ford_mondeo_16 mary");
s.printOntology();

#### 2e. Creating TBox axioms

In [ ]:
// TBox axioms
// create subclass axioms
s.createAxiom("Woman subClassOf Person and (hasGender some Female)"); 
s.createAxiom("Man subClassOf Person and (hasGender some Male)");
s.createAxiom("Car subClassOf hasPart exactly 4 Tyre");
s.createAxiom("USCitizen subClassOf hasDocument max 2 Passport");
s.createAxiom("Arthropod subClassOf hasPart min 6 Leg");
// create OWL equivalent classes axiom and add it to the currently selected ontology
s.createAxiom("Man equivalentTo Person and Adult and (hasGender some Male)"); 
s.createAxiom("Woman equivalentTo Person and Adult and (hasGender some Female)"); 
// create OWL disjoint classes axiom and add it to the currently selected ontology
s.createAxiom("Man disjointWith Woman"); 
s.createAxiom("Female disjointWith Male");
// create axioms using nominals
s.createAxiom("JohnsBookClubMembers equivalentTo {john,mary,sara}");
// create domain & range axioms
s.createAxiom("hasGender Domain: Person");
s.createAxiom("hasGender Range: Male or Female");
// print the ontology
s.printOntology();

#### 2f. Creating ABox axioms

In [ ]:
// ABox axioms
// create class assertion
s.createAxiom("john Type: Adult");
s.createAxiom("mary Type: Adult");
// **NB: create property assertion**
// With property assertions the method name is different
// **we cannot use createAxiom here**
s.createObjectPropertyAssertion("john hasGender Male");
s.createObjectPropertyAssertion("mary hasGender Female");
s.createObjectPropertyAssertion("john marriedTo mary");
// Create different individuals axiom (specify each individual name)
s.differentIndividuals("john mary sara");
// Make **all** individual names in the ontology refer to distinct objects from the domain
s.allIndividualsDifferent();
s.printOntology();

#### 2g. Creating RBox axioms

In [ ]:
// RBox axioms
// **note the ":" after SubPropertyOf and InverseOf**
s.createAxiom("hasSister SubPropertyOf: hasSibling");
// role / property composition axiom
s.createAxiom("owns o hasPart SubPropertyOf: owns");
// inverse property axiom
s.createAxiom("teaches InverseOf: studentOf");
// print ontology 
s.printOntology();
// print some stats about the ontology (e.g. # of classes, properties etc.)
s.printOntologyStats(); 

#### 2h. Saving an ontology to file; loading an ontology from a remote IRI; and removing entities from an ontology

In [ ]:
// save active ontology to local file using Manchester OWL syntax
s.saveOntology("testontology.owl"); 
// load an ontology into this context from a remote URL
//s.loadFromURL("https://protege.stanford.edu/ontologies/pizza/pizza.owl"); 
// load an ontology into this context from a local file path. WARNING: you cannot load multiple ontologies with the same IRI into the same context!
//s.loadFromFile("path/to/ontology.owl");
// remove selected ontology from current context (simpleOWLAPIFactory instance) 
//s.removeOntology(); 
// remove ontology with specified IRI from current context (simpleOWLAPIFactory instance) 
//s.removeOntology("http://iri-of-ontology"); 

// remove all axioms from the currently selected ontology
//s.resetOntology(); 
// remove multiple class names from currently selected ontology
//s.removeClasses("Bird Penguin"); 
// remove multiple object properties from currently selected ontology
//s.removeObjectProperties("knows hasPart"); 
// remove multiple data properties from currently selected ontology
//s.removeDataProperties("hasWeight name"); 
// remove multiple individual names from currently selected ontology
//s.removeIndividuals("tweety woody"); 
// remove an axiom from the currently selected ontology
//s.removeAxiom("Penguin subClassOf eats some Fish"); 

#### 2i. Visualise your ontology using [WebVowel](http://webvowl.137.120.31.101.nip.io/)
You can upload your OWL file (less than 5MB) to the WebVowel online ontology visualiser to explore your ontology graphically. Just in case the link above does not work for WebVowel, you can try [this one](http://webvowl.semanticscience.org) as well. Protégé itself also has ontology visualisation plugins e.g. [OntoGraf](https://github.com/protegeproject/ontograf) developed by its users and community. You can find OntoGraf under Window>Tabs>OntoGraf in Protégé. However, WebVowel has a more visually appealing interface. There are other ontology visualisers available as well, for example, [OWLGrEd](http://owlgred.lumii.lv/online_visualization). Feel free to try these tools out, they are quite helpful to get an overview of the kind of knowledge present in an ontology.

![title](img/visualise_your_ont.png)

#### 3. Reasoning with your ontology using OWL 2 DL reasoners
Here we will use [HermiT](http://www.hermit-reasoner.com/) which is a robust OWL 2 DL reasoner. There is also [Pellet](https://github.com/stardog-union/pellet) and [Fact++](https://bitbucket.org/dtsarkov/factplusplus/downloads/uk.ac.manchester.cs.owl.factplusplus-P5.x-v1.6.5.jar) as alternatives amongst [others](https://www.w3.org/2001/sw/wiki/OWL/Implementations)

#### 3a. Compute the subclasses or superclasses of a given class

In [ ]:
// Get super classes of a given class expression / atomic class
s.owlReasoner.getSuperClasses("hasGender some Male");
// Get subclasses of a given class expression / atomic class
s.owlReasoner.getSubClasses("hasGender some Male");

#### 3b. Is the ontology consistent?

In [ ]:
s.createClasses("Bird Penguin FlyingOrganism");
s.createAxiom("Bird subClassOf FlyingOrganism");
s.createAxiom("Penguin subClassOf Bird");
s.createAxiom("Penguin subClassOf not FlyingOrganism");
s.createIndividual("tweety");
s.createAxiom("tweety Type: Penguin");
// switch or set OWL reasoner
//s.setOWLReasoner(SelectedReasoner.HERMIT);
// prints to console Yes if currently selected ontology is consistent, No otherwise
s.owlReasoner.isConsistent();
// computes and prints to console all explanations for the inconsistency of the selected ontology (provided it is inconsistent)
s.owlReasoner.explainInconsistency();

#### 3c. Check unsatisfiability of a class

In [ ]:
// first remove this axiom which causes the inconsistency
s.removeAxiom("tweety Type: Penguin");
// now check again if the ontology is consistent.
s.owlReasoner.isConsistent();
// compute all unsatisfiable atomic classes in the ontology
s.owlReasoner.isSatisfiable("Penguin");

#### 3d. Check if a subsumption holds

In [ ]:
// Check if some axiom logically follows from the ontology
s.owlReasoner.isEntailed("Man subClassOf Bird");

#### 3e. ABox realisation (compute class membership of individuals and relations between them)

In [ ]:
// Get all atomic classes which this individual belongs to
s.owlReasoner.getTypes("mary");
// Get all atomic classes for each individual in the ontology
s.owlReasoner.getAllTypes();
// Get all instances of a class
s.owlReasoner.getInstances("Person");
// Get all property assertions for the given property
s.owlReasoner.getObjectPropertyAssertions("marriedTo");
// Get all property assertions in the ontology
s.owlReasoner.getAllObjectPropertyAssertions();

#### 3f. Compute justifications (explanations) for a given inference

In [ ]:
// explain unsatisfiability of a class (if it is unsatisfiable)
s.owlReasoner.explainUnsatisfiability("Penguin");
// explain general entailment (if this entailment does hold)
s.owlReasoner.explainEntailment("john Type: Person");

#### 4. Start of lab (build your family relations ontology here)

In [ ]:
// Start building your ontology here //